In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

IMPORT DATA SET

In [2]:
# Cargar el conjunto de datos
url = 'https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv'

In [3]:
df = pd.read_csv(url)
print(df.head())


   rownames       wage     lwage  sex  shs  hsg  scl  clg   ad   mw  ...   we  \
0        10   9.615385  2.263364  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
1        12  48.076923  3.872802  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
2        15  11.057692  2.403126  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0   
3        18  13.942308  2.634928  1.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0   
4        19  28.846154  3.361977  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   

    ne  exp1  exp2    exp3     exp4     occ  occ2     ind  ind2  
0  1.0   7.0  0.49   0.343   0.2401  3600.0    11  8370.0    18  
1  1.0  31.0  9.61  29.791  92.3521  3050.0    10  5070.0     9  
2  1.0  18.0  3.24   5.832  10.4976  6260.0    19   770.0     4  
3  1.0  25.0  6.25  15.625  39.0625   420.0     1  6990.0    12  
4  1.0  22.0  4.84  10.648  23.4256  2015.0     6  9470.0    22  

[5 rows x 21 columns]


In [4]:
# Asegurarse que los nombres de las variables están bien cargados
print(df.columns)

Index(['rownames', 'wage', 'lwage', 'sex', 'shs', 'hsg', 'scl', 'clg', 'ad',
       'mw', 'so', 'we', 'ne', 'exp1', 'exp2', 'exp3', 'exp4', 'occ', 'occ2',
       'ind', 'ind2'],
      dtype='object')


sex, 1 equals Female
shs: "some highschool"
hsg: "highschool graduate"
scl: "some college"
clg: "college graduate"
ad: "advanced degree"
mw: "midwest"
so: "south"
we: "west"
ne: "northeast"
occ2: "occupation code"
ind2: "industry code"

MISSING VALUES

In [5]:
# Revisar los missing values
missing_values = df.isnull().sum()
print("Valores faltantes:", missing_values)

Valores faltantes: rownames    0
wage        0
lwage       0
sex         0
shs         0
hsg         0
scl         0
clg         0
ad          0
mw          0
so          0
we          0
ne          0
exp1        0
exp2        0
exp3        0
exp4        0
occ         0
occ2        0
ind         0
ind2        0
dtype: int64


In [6]:
# Estadísticas descriptivas
descriptive_stats = df.describe(percentiles=[0.25, 0.5, 0.75])
print("Estadísticas descriptivas:", descriptive_stats)

Estadísticas descriptivas:            rownames         wage        lwage          sex          shs  \
count   5150.000000  5150.000000  5150.000000  5150.000000  5150.000000   
mean   15636.346602    23.410410     2.970787     0.444466     0.023301   
std     9700.744395    21.003016     0.570385     0.496955     0.150872   
min       10.000000     3.021978     1.105912     0.000000     0.000000   
25%     7132.500000    13.461538     2.599837     0.000000     0.000000   
50%    15260.000000    19.230769     2.956512     0.000000     0.000000   
75%    24337.750000    27.777778     3.324236     1.000000     0.000000   
max    32643.000000   528.845673     6.270697     1.000000     1.000000   

               hsg          scl          clg           ad           mw  ...  \
count  5150.000000  5150.000000  5150.000000  5150.000000  5150.000000  ...   
mean      0.243883     0.278058     0.317670     0.137087     0.259612  ...   
std       0.429465     0.448086     0.465616     0.343973   

In [6]:
#WAGE: La media del salario es 23.41, con una desviación estándar de 21.08, lo que indica que hay una gran variabilidad en los salarios. 
# El salario mediano es 19.23, lo que significa que el 50% de los sujetos ganan menos de esta cantidad, 
# lo que indica una distribución sesgada positivamente, ya que la media es mayor que la mediana. 
# El salario máximo es bastante alto en comparación con la mayoría de los salarios, lo que sugiere la presencia de valores atípicos.

In [7]:
#lwage: se usa para normalizar la distribución de salarios. 
#Los percentiles son similares a los del salario, pero en una escala logarítmica, lo que ayuda a reducir el impacto de valores extremos

In [8]:
#sex: Con una media de 0.444, aproximadamente el 44.4% de los individuos en la muestra son mujeres (si 1 = mujer). 
# La desviación estándar cercana a 0.5 es típica para una variable binaria con proporciones relativamente equilibradas

In [9]:
#shs: solo el 2.3% de la muestra ha completado "algo de secundaria"
#hsg:  el 24.4% están graduados de la escuela secundaria
#scl: 27.8% de la muestra ha completado "algo de la universidad"
#clg: 31.8% son graduados universitarios
#ad: 13.7% tienen un titulo avanzado
#mw: 25.9% pertenen al medio oeste
#we: 21.6% son parte del sur
#ne: 22.7% del norte

In [12]:
#mujeres con título universitario o superior tienen un salario correspondiente al 25% más rico de la muestra

# Filtrar las mujeres con título universitario o superior
mujeres_tituladas_superior_df = df[(df['sex'] == 1) & ((df['clg'] == 1) | (df['ad'] == 1))]

# Determinar el umbral del 25% más rico para toda la muestra
salariado_75 = df['wage'].quantile(0.75)

# Filtrar las mujeres con salario en el 25% más alto
mujeres_mas_asalariadas_df = mujeres_tituladas_superior_df[mujeres_tituladas_superior_df['wage'] >= salariado_75]


print(mujeres_mas_asalariadas_df)

      rownames        wage     lwage  sex  shs  hsg  scl  clg   ad   mw  ...  \
4           19   28.846154  3.361977  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...   
30         191   42.307692  3.744969  1.0  0.0  0.0  0.0  0.0  1.0  0.0  ...   
39         232   41.208791  3.718652  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...   
54         319  100.000000  4.605170  1.0  0.0  0.0  0.0  0.0  1.0  0.0  ...   
91         563   33.653846  3.516127  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...   
...        ...         ...       ...  ...  ...  ...  ...  ...  ...  ...  ...   
5085     32251   48.076923  3.872802  1.0  0.0  0.0  0.0  0.0  1.0  0.0  ...   
5093     32314   30.145530  3.406037  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...   
5105     32419   29.914530  3.398344  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...   
5131     32567   48.076923  3.872802  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...   
5140     32596   45.546559  3.818735  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...   

       we   ne  exp1   exp2    exp3    

In [13]:
# Contar cuántas mujeres cumplen con ambas condiciones
contar_mujeres = mujeres_mas_asalariadas_df.shape[0]

print(contar_mujeres)

419


In [14]:
#hombres con título de bachillerato o inferior tienen un salario correspondiente al 25 % más rico de la muestra

#hombres con bachillerato o inferor
hombres_bachillerato_inferior_df =df[(df['sex'] == 0) & ((df['hsg'] == 1)|(df["shs"]==1))]

# Filtrar los hombres con salario en el 25% más alto
hombres_mas_asalariados_df = hombres_bachillerato_inferior_df[hombres_bachillerato_inferior_df['wage'] >= salariado_75]

print(hombres_mas_asalariados_df)


      rownames       wage     lwage  sex  shs  hsg  scl  clg   ad   mw  ...  \
15         113  27.884615  3.328075  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
46         276  28.846154  3.361977  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
75         467  28.846154  3.361977  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
136        858  28.846154  3.361977  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
140        876  29.714286  3.391628  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
...        ...        ...       ...  ...  ...  ...  ...  ...  ...  ...  ...   
5074     32191  33.653846  3.516127  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
5086     32254  40.865385  3.710283  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
5089     32281  31.250000  3.442019  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
5094     32321  33.653846  3.516127  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
5148     32631  32.967033  3.495508  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   

       we   ne  exp1   exp2    exp3      exp4     o

In [15]:
# Contar cuántos hombres cumplen con ambas condiciones
contar_hombres = hombres_mas_asalariados_df.shape[0]

print(contar_hombres)


118


In [16]:
# Crear un marco de datos con solo la variable 'wage'
lwage_df = df[['lwage']]

# Crear un marco de datos con todas las variables excepto 'wage'
datos_df = df.drop(columns=['wage','lwage'])

In [17]:
print(lwage_df)
print(datos_df)

         lwage
0     2.263364
1     3.872802
2     2.403126
3     2.634928
4     3.361977
...        ...
5145  2.692546
5146  3.138833
5147  3.649659
5148  3.495508
5149  2.851151

[5150 rows x 1 columns]
      rownames  sex  shs  hsg  scl  clg   ad   mw   so   we   ne  exp1  exp2  \
0           10  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0   7.0  0.49   
1           12  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  31.0  9.61   
2           15  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  18.0  3.24   
3           18  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  25.0  6.25   
4           19  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  22.0  4.84   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
5145     32620  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   9.0  0.81   
5146     32624  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  12.0  1.44   
5147     32626  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  11.0  1.2

DATA WRANGLING

Modelo Básico

In [56]:
# Crear variables para regresión
# a) Variable dependiente
Y = df['lwage']

In [57]:
# b) Predictores modelo básico
X_basic = df[['sex', 'exp1', 'hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne']]


In [58]:
# Convertir occ2 e ind2 a variables dummy
X_basic = pd.concat([X_basic, pd.get_dummies(df['occ2'], drop_first=True), pd.get_dummies(df['ind2'], drop_first=True)], axis=1)


Modelo Flexible 

In [59]:
# Modelo flexible: añadir polinomios e interacciones de experiencia
df['exp2'] = df['exp1'] ** 2
df['exp3'] = df['exp1'] ** 3
df['exp4'] = df['exp1'] ** 4


In [60]:
X_flexible = X_basic.copy()
X_flexible['exp2'] = df['exp2']
X_flexible['exp3'] = df['exp3']
X_flexible['exp4'] = df['exp4']


In [61]:
# Crear interacciones
interaction_vars = ['hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne']
for exp_var in ['exp1', 'exp2', 'exp3', 'exp4']:
    for var in interaction_vars:
        X_flexible[f'{exp_var}*{var}'] = df[exp_var] * df[var]


Modelo Extraflexible

In [62]:
# Modelo extraflexible: todas las interacciones bidireccionales
X_extraflexible = X_flexible.copy()


In [63]:
import itertools


In [66]:
# Restablecer los índices para evitar duplicados
X_flexible = X_flexible.reset_index(drop=True)
X_extraflexible = X_extraflexible.reset_index(drop=True)

# Crear combinaciones de pares de columnas sin repeticiones
for var1, var2 in itertools.combinations(X_flexible.columns, 2):
    col_name = f'{var1}*{var2}'
    if col_name not in X_extraflexible.columns:
        # Verificar si las columnas son multidimensionales y seleccionar la primera dimensión si es el caso
        if len(X_flexible[var1].shape) > 1:
            column1 = X_flexible[var1].iloc[:, 0]  # Seleccionar la primera subcolumna
        else:
            column1 = X_flexible[var1]

        if len(X_flexible[var2].shape) > 1:
            column2 = X_flexible[var2].iloc[:, 0]  # Seleccionar la primera subcolumna
        else:
            column2 = X_flexible[var2]

        # Multiplicar columnas de la misma dimensión
        X_extraflexible[col_name] = column1 * column2


C:\Users\Personal\AppData\Local\Temp\ipykernel_11392\2716754080.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extraflexible[col_name] = column1 * column2
C:\Users\Personal\AppData\Local\Temp\ipykernel_11392\2716754080.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extraflexible[col_name] = column1 * column2
C:\Users\Personal\AppData\Local\Temp\ipykernel_11392\2716754080.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

In [67]:
# Dividir en entrenamiento y prueba
X_train_basic, X_test_basic, Y_train, Y_test = train_test_split(X_basic, Y, test_size=0.2, random_state=42)
X_train_flexible, X_test_flexible, _, _ = train_test_split(X_flexible, Y, test_size=0.2, random_state=42)
X_train_extraflexible, X_test_extraflexible, _, _ = train_test_split(X_extraflexible, Y, test_size=0.2, random_state=42)


In [69]:
# Ajustar modelos y calcular métricas
def fit_and_evaluate(X_train, X_test, Y_train, Y_test):
    model = LinearRegression()
    model.fit(X_train, Y_train)
    
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    
    mse_train = mean_squared_error(Y_train, Y_pred_train)
    mse_test = mean_squared_error(Y_test, Y_pred_test)
    
    r2_train = r2_score(Y_train, Y_pred_train)
    r2_test = r2_score(Y_test, Y_pred_test)
    
    return mse_train, mse_test, r2_train, r2_test




In [72]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression


In [77]:
# Convertir los nombres de las columnas a cadenas en X_train_basic y X_test_basic
X_train_basic.columns = X_train_basic.columns.astype(str)
X_test_basic.columns = X_test_basic.columns.astype(str)

# Llamada a la función fit_and_evaluate para el modelo básico
mse_train_basic, mse_test_basic, r2_train_basic, r2_test_basic = fit_and_evaluate(X_train_basic, X_test_basic, Y_train, Y_test)



In [78]:
# Convertir los nombres de las columnas a cadenas en X_train_flexible y X_test_flexible (si es necesario)
X_train_flexible.columns = X_train_flexible.columns.astype(str)
X_test_flexible.columns = X_test_flexible.columns.astype(str)

# Llamada a la función fit_and_evaluate para el modelo flexible
mse_train_flexible, mse_test_flexible, r2_train_flexible, r2_test_flexible = fit_and_evaluate(X_train_flexible, X_test_flexible, Y_train, Y_test)

In [80]:
# Convertir los nombres de las columnas a cadenas en X_train_flexible y X_test_flexible (si es necesario)
X_train_extraflexible.columns = X_train_extraflexible.columns.astype(str)
X_test_extraflexible.columns = X_test_extraflexible.columns.astype(str)

# Llamada a la función fit_and_evaluate para el modelo flexible
mse_train_extraflexible, mse_test_extraflexible, r2_train_extraflexible, r2_test_extraflexible = fit_and_evaluate(X_train_extraflexible, X_test_extraflexible, Y_train, Y_test)

In [81]:
# Imprimir resultados de los tres modelos
print(f"Modelo Básico - MSE Train: {mse_train_basic}, MSE Test: {mse_test_basic}, R² Train: {r2_train_basic}, R² Test: {r2_test_basic}")
print(f"Modelo Flexible - MSE Train: {mse_train_flexible}, MSE Test: {mse_test_flexible}, R² Train: {r2_train_flexible}, R² Test: {r2_test_flexible}")
print(f"Modelo Extraflexible - MSE Train: {mse_train_extraflexible}, MSE Test: {mse_test_extraflexible}, R² Train: {r2_train_extraflexible}, R² Test: {r2_test_extraflexible}")



Modelo Básico - MSE Train: 0.2222465683493698, MSE Test: 0.23636335287368276, R² Train: 0.3154005637177951, R² Test: 0.2789935069708511
Modelo Flexible - MSE Train: 0.21923222710629245, MSE Test: 0.2330217143195311, R² Train: 0.3246858198686525, R² Test: 0.2891868938288701
Modelo Extraflexible - MSE Train: 0.5166110736616618, MSE Test: 39061.880798785875, R² Train: -0.591348079894531, R² Test: -119153.97619847431
